In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb

In [ ]:
df_train=pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')
train_target=df_train['target']
train_drop=['target','lesion_id','iddx_full','iddx_1',
            'iddx_2','iddx_3','iddx_4','iddx_5','mel_mitotic_index',
            'mel_thick_mm','tbp_lv_dnn_lesion_confidence','isic_id','patient_id']
df_train.drop(train_drop,axis=1,inplace=True)
temp=df_train['age_approx'].median()
df_train['age_approx']=df_train['age_approx'].fillna(temp)
df_train.dropna(axis=1,inplace=True)

In [ ]:
df_test=pd.read_csv('/kaggle/input/isic-2024-challenge/test-metadata.csv')
df_test['age_approx']=df_test['age_approx'].fillna(temp)
isic_id=df_test['isic_id']
for col_test in df_test.columns:
    g=1
    for col in df_train.columns:
        if col_test==col:
            g=0
    if g==1:
        df_test.drop(col_test,axis=1,inplace=True)

In [ ]:
lb=LabelEncoder()

In [ ]:
i=0
cat_features=[]
label_features=[]
for col in df_train.columns:   
    if df_train[col].dtype=='O':
        cat_features.append(i)
        df_train[col]=lb.fit_transform(df_train[col])
        df_test[col]=lb.fit_transform(df_test[col])
    i+=1

In [ ]:
clf = lgb.LGBMClassifier(objective=        'binary',
                         verbosity=        -1,
                         n_iter=           200,
                         boosting_type=    'gbdt',
                         random_state=     3,
                         lambda_l1=        0.08758718919397321, 
                         lambda_l2=        0.0039689175176025465, 
                         learning_rate=    0.03231007103195577, 
                         max_depth=        4, 
                         num_leaves=       103, 
                         colsample_bytree= 0.8329551585827726, 
                         colsample_bynode= 0.4025961355653304, 
                         bagging_fraction= 0.7738954452473223, 
                         bagging_freq=     4, 
                         min_data_in_leaf= 85, 
                         scale_pos_weight= 2.7984184778875543
                        )
clf.fit(df_train, train_target,categorical_feature=cat_features)

In [ ]:
pred=clf.predict_proba(df_test)
pred=pd.DataFrame(pred[:,1])
df_sub=pd.concat([isic_id,pred],axis=1)
df_sub.to_csv('/kaggle/working/submission.csv',index=False)

In [ ]:
df_sub